In [27]:
%reset -f

In [28]:
import pandas as pd
from sqlalchemy import create_engine, inspect, text
from datetime import datetime

# Configuration — adjust to your local PostgreSQL setup
DB_NAME         = "sandbox"
DB_USER         = "postgres"
DB_PASSWORD     = "AsidDe5845edDikkDee"
DB_HOST         = "localhost"
DB_PORT         = "5432"

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Define your table names (no backticks needed in PostgreSQL)
schema_games        = "stg_chess_com"
table_games         = "players_games"
schema_games_times  = "stg_times"
table_games_times   = "games_times"

# Helper: check if table with prefix exists
def table_with_prefix_exists(engine, schema_name, table_prefix):
    inspector = inspect(engine)
    tables = inspector.get_table_names(schema=schema_name)
    return any(t.startswith(table_prefix) for t in tables)

# Choose query based on table existence
if table_with_prefix_exists(engine, schema_games_times, schema_games_times):
    query = f"""
    SELECT
        game.uuid,
        MAX(game.pgn) AS pgn
    FROM {schema_games}.{table_games} game
    LEFT JOIN (
        SELECT DISTINCT uuid FROM {schema_games_times}.{schema_games_times} games_moves
    ) games_moves
    ON game.uuid = games_moves.uuid
    WHERE 
        games_moves.uuid IS NULL
        AND LENGTH(game.pgn) > 0
        AND game.rules = 'chess'
    GROUP BY 1
    """
else:
    query = f"SELECT uuid, pgn FROM {schema_games}.{table_games}"

# Run query and load into DataFrame
games = pd.read_sql(query, engine)
print("Query to fetch games executed successfully!")

Query to fetch games executed successfully!


In [31]:
import pandas as pd
import re

def extract_move_data(pgn):
    clocks = re.findall(r'\[%clk (\d+):(\d{2}):(\d{2}(?:\.\d)?)\]', pgn)
    return [
        {
            'move_number': i + 1,
            'time_remaining_seconds': int(h) * 3600 + int(m) * 60 + float(s),
            'time_remaining': f"{h}:{m}:{s}"
        }
        for i, (h, m, s) in enumerate(clocks)
    ]

games['move_data'] = games['pgn'].apply(extract_move_data)
games_expanded = games.explode('move_data', ignore_index=True)

games_expanded[['move_number', 'time_remaining_seconds', 'time_remaining']] = pd.json_normalize(games_expanded['move_data'])
games_expanded["log_timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
games_expanded = games_expanded.drop(columns=['move_data', 'pgn'])

In [32]:
with engine.begin() as conn:
    conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {schema_games_times}"))

games_expanded.to_sql(
    name        = table_games_times,
    con         = engine,
    schema      = schema_games_times,     
    if_exists   = 'append', # If the table exists
    index       = False # Ignore the df index   
)

94